<a href="https://colab.research.google.com/github/kairamilanifitria/Bootcamp-NLP/blob/main/Advanced/23102024_Train_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!mkdir -p /content/gpt2-custom

In [3]:
! pip install accelerate -U
! pip install tokenizers
! pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2


In [4]:
!CUDA_LAUNCH_BLOCKING=1

In [5]:
from tokenizers import ByteLevelBPETokenizer

paths = ["/content/drive/MyDrive/Bootcamp AI/Dataset/clean_id_100k.txt"]

tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=paths, vocab_size=10000)
tokenizer.save_model("/content/gpt2-custom")

['/content/gpt2-custom/vocab.json', '/content/gpt2-custom/merges.txt']

In [6]:
teks = "saya mau permakanan nasi"
enc = tokenizer.encode(teks)
dec = tokenizer.decode(enc.ids)

In [7]:
print(f"Teks: {teks}")
print(f"Token IDs: {enc.ids}")
print(f"Token words: {enc.tokens}")
print(f"Token decode: {dec}")

Teks: saya mau permakanan nasi
Token IDs: [82, 447, 1386, 1569, 310, 256, 4283]
Token words: ['s', 'aya', 'Ġmau', 'Ġperm', 'akan', 'an', 'Ġnasi']
Token decode: saya mau permakanan nasi


In [8]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments, GPT2Config

In [9]:
# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("/content/gpt2-custom")
config = GPT2Config(vocab_size=tokenizer.vocab_size)
model = GPT2LMHeadModel(config=config)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
# Load and tokenize dataset
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=paths[0],
    block_size=128,  # Adjust as needed
)
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [11]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-custom",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

# Create Trainer and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [13]:
import wandb
wandb.init(mode="disabled")

In [14]:
%%time
trainer.train()
trainer.save_model('/content/gpt2-custom')

Step,Training Loss
500,7.066100
1000,6.436300
1500,6.146300
2000,5.939600
2500,5.775000
3000,5.612800
3500,5.520700
4000,5.462700
4500,5.422200


CPU times: user 18min 12s, sys: 7.27 s, total: 18min 19s
Wall time: 18min 43s


In [15]:
from transformers import pipeline

generator = pipeline("text-generation", model="./gpt2-custom", tokenizer=tokenizer)
generated_text = generator("Partai ini berhasil")
print(generated_text)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


[{'generated_text': 'Partai ini berhasil melakukan tes swab.\nPasar ini, kata dia, dia, dia,'}]
